<a href="https://colab.research.google.com/github/theAbyssOfTime2004/cat-classification/blob/main/notebooks/model_definition_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/theAbyssOfTime2004/cat-classification.git

Cloning into 'cat-classification'...
remote: Enumerating objects: 13608, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 13608 (delta 2), reused 2 (delta 2), pack-reused 13517 (from 2)
Receiving objects: 100% (13608/13608), 165.03 MiB | 26.18 MiB/s, done.
Resolving deltas: 100% (722/722), done.
Updating files: 100% (30986/30986), done.
Filtering content: 100% (4/4), 799.86 MiB | 50.39 MiB/s, done.


In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2 # THAY ĐỔI: EfficientNetB0 -> ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input # THÊM MỚI
from tensorflow.keras.models import Model # THAY ĐỔI: Sequential -> Model (cho Functional API)
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten # THÊM Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# --- Constants ---
PROCESSED_DATA_DIR = '/content/drive/MyDrive/processed/processed' # Đường dẫn mới trên Google Drive
TRAIN_DIR = os.path.join(PROCESSED_DATA_DIR, 'train')
VAL_DIR = os.path.join(PROCESSED_DATA_DIR, 'val')
TEST_DIR = os.path.join(PROCESSED_DATA_DIR, 'test')

# Model and training parameters
IMG_WIDTH, IMG_HEIGHT = 224, 224 # Kích thước ảnh đầu vào cho ResNet50V2
BATCH_SIZE = 32
EPOCHS = 50 # Số epochs ban đầu, có thể điều chỉnh
LEARNING_RATE = 1e-4

# Xác định số lớp (số giống mèo)
if not os.path.exists(TRAIN_DIR):
    raise FileNotFoundError(f"Thư mục huấn luyện không tìm thấy tại: {TRAIN_DIR}. Hãy chạy notebook data_splitting trước.")

num_classes = len([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
print(f"Tìm thấy {num_classes} lớp (giống mèo).")

if num_classes == 0:
    raise ValueError(f"Không tìm thấy thư mục con nào trong {TRAIN_DIR}. Kiểm tra lại dữ liệu đã xử lý.")

Tìm thấy 63 lớp (giống mèo).


In [10]:
# --- Data Generators ---
# ImageDataGenerator cho tập huấn luyện với tăng cường dữ liệu
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ImageDataGenerator cho tập validation và test (chỉ chuẩn hóa, không tăng cường)
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # THAY ĐỔI

# Tạo generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical', # Vì chúng ta có nhiều lớp
    shuffle=True
)

validation_generator = val_test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Không cần shuffle tập validation
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Không cần shuffle tập test
)

# Kiểm tra xem generator có hoạt động không
try:
    sample_batch_train, sample_labels_train = next(train_generator)
    print(f"Hình dạng batch huấn luyện: {sample_batch_train.shape}")
    print(f"Hình dạng nhãn huấn luyện: {sample_labels_train.shape}")

    sample_batch_val, sample_labels_val = next(validation_generator)
    print(f"Hình dạng batch validation: {sample_batch_val.shape}")
    print(f"Hình dạng nhãn validation: {sample_labels_val.shape}")
except Exception as e:
    print(f"Lỗi khi tạo batch mẫu từ generator: {e}")
    print("Hãy kiểm tra lại đường dẫn TRAIN_DIR, VAL_DIR và cấu trúc thư mục con.")


Found 5877 images belonging to 63 classes.
Found 1260 images belonging to 63 classes.
Found 1260 images belonging to 63 classes.
Hình dạng batch huấn luyện: (32, 224, 224, 3)
Hình dạng nhãn huấn luyện: (32, 63)
Hình dạng batch validation: (32, 224, 224, 3)
Hình dạng nhãn validation: (32, 63)


In [11]:
# --- Model Definition (Transfer Learning with ResNet50V2) ---

# 1. Tải base model (ResNet50V2) đã được huấn luyện trước trên ImageNet
base_model = ResNet50V2(
    weights='imagenet',
    include_top=False, # Loại bỏ lớp phân loại cuối cùng
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
)

# 2. Đóng băng các trọng số của base model
base_model.trainable = False

# 3. Xây dựng mô hình mới bằng cách thêm các lớp tùy chỉnh lên trên base_model (Functional API)
# Lấy đầu ra của base_model
x = base_model.output

# Thêm các lớp tùy chỉnh
x = Flatten()(x) # Sử dụng Flatten thay vì GlobalAveragePooling2D như trong đoạn code mới của bạn
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
# Lớp đầu ra với num_classes (đã xác định ở ô đầu tiên) và softmax
predictions = Dense(num_classes, activation='softmax')(x)

# Tạo mô hình hoàn chỉnh
model = Model(inputs=base_model.input, outputs=predictions)

# In cấu trúc mô hình
model.summary()

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, 56, 56,    │          0 │ conv2_block1_pre… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, 58, 58,    │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, 56, 56,    │          0 │ conv2_block1_0_c

 Total params: 74,977,855 (286.02 MB)

 Trainable params: 51,413,055 (196.13 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [12]:
# --- Model Compilation ---
optimizer = Adam(learning_rate=LEARNING_RATE) # LEARNING_RATE = 1e-4 từ ô đầu tiên

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy', # Giữ nguyên vì class_mode='categorical'
    metrics=['accuracy'] # Theo dõi độ chính xác ('acc' và 'accuracy' là như nhau)
)
print("Đã biên dịch mô hình.")

Đã biên dịch mô hình.


In [13]:
# --- Callbacks ---
# ModelCheckpoint: Lưu lại mô hình tốt nhất dựa trên val_accuracy
checkpoint_filepath = os.path.expanduser('/content/drive/MyDrive/cat-classification/models/cat_breed_classifier_best.keras')
os.makedirs(os.path.dirname(checkpoint_filepath), exist_ok=True) # Tạo thư mục models nếu chưa có

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, # Lưu toàn bộ mô hình
    monitor='val_accuracy',
    mode='max',
    save_best_only=True, # Chỉ lưu mô hình nếu val_accuracy cải thiện
    verbose=1
)

# EarlyStopping: Dừng huấn luyện sớm nếu không có cải thiện
early_stopping_callback = EarlyStopping(
    monitor='val_loss', # Theo dõi val_loss
    patience=10,        # Số epochs không cải thiện trước khi dừng
    verbose=1,
    restore_best_weights=True # Khôi phục trọng số từ epoch tốt nhất
)

# ReduceLROnPlateau: Giảm learning rate khi không có cải thiện
reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, # learning_rate sẽ nhân với factor này
    patience=5,
    min_lr=1e-6, # Learning rate tối thiểu
    verbose=1
)

callbacks_list = [model_checkpoint_callback, early_stopping_callback, reduce_lr_callback]

In [14]:
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"TensorFlow has access to the following GPU(s): {gpu_devices}")
    # You can also enable device placement logging in TensorFlow
    # tf.debugging.set_log_device_placement(True)
    # to see where operations are being placed.
else:
    print("TensorFlow does not have access to a GPU. Training will run on the CPU.")

TensorFlow has access to the following GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [15]:
# --- Model Training ---
print("Bắt đầu huấn luyện mô hình...")

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks_list,
    steps_per_epoch=train_generator.samples // BATCH_SIZE, # Số batch mỗi epoch
    validation_steps=validation_generator.samples // BATCH_SIZE # Số batch cho validation
)

print("Hoàn tất huấn luyện mô hình (giai đoạn đầu - feature extraction).")


Bắt đầu huấn luyện mô hình...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 542ms/step - accuracy: 0.0443 - loss: 5.5402
Epoch 1: val_accuracy improved from -inf to 0.10978, saving model to /content/drive/MyDrive/cat-classification/models/cat_breed_classifier_best.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 130s 629ms/step - accuracy: 0.0444 - loss: 5.5345 - val_accuracy: 0.1098 - val_loss: 3.7018 - learning_rate: 1.0000e-04
Epoch 2/50
  1/183 ━━━━━━━━━━━━━━━━━━━━ 18s 100ms/step - accuracy: 0.0625 - loss: 3.8278

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy improved from 0.10978 to 0.11298, saving model to /content/drive/MyDrive/cat-classification/models/cat_breed_classifier_best.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 12s 65ms/step - accuracy: 0.0625 - loss: 3.8278 - val_accuracy: 0.1130 - val_loss: 3.6945 - learning_rate: 1.0000e-04
Epoch 3/50
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 515ms/step - accuracy: 0.0823 - loss: 3.8802
Epoch 3: val_accuracy improved from 0.11298 to 0.14904, saving model to /content/drive/MyDrive/cat-classification/models/cat_breed_classifier_best.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 107s 586ms/step - accuracy: 0.0823 - loss: 3.8802 - val_accuracy: 0.1490 - val_loss: 3.4535 - learning_rate: 1.0000e-04
Epoch 4/50
  1/183 ━━━━━━━━━━━━━━━━━━━━ 17s 98ms/step - accuracy: 0.0938 - loss: 3.9033
Epoch 4: val_accuracy improved from 0.14904 to 0.14984, saving model to /content/drive/MyDrive/cat-classification/models/cat_breed_classifier_best.keras
183/183 ━━━━━━━━━━━━━━━━━━━━ 11s 62ms/step - accuracy: 0.0938 - loss